In [0]:
from pyspark import *
from pyspark.sql import SparkSession, Window
from pyspark.sql.types import StructType,StructField, StringType, IntegerType, DateType, TimestampType, LongType
from pyspark.sql.types import ArrayType, DoubleType, BooleanType, DecimalType
from pyspark.sql.functions import regexp_extract, split, from_unixtime, col, avg, min, max, desc
from pyspark.sql.functions import grouping, explode, array_contains, struct, collect_list, row_number

In [0]:
%fs ls dbfs:/FileStore/tables

path,name,size,modificationTime
dbfs:/FileStore/tables/PADRON_COMPLETO-1.csv,PADRON_COMPLETO-1.csv,437542254,1749404416000
dbfs:/FileStore/tables/PADRON_COMPLETO.csv,PADRON_COMPLETO.csv,437542254,1748737791000
dbfs:/FileStore/tables/SJ.csv,SJ.csv,140579880,1748917446000
dbfs:/FileStore/tables/distelec.csv,distelec.csv,175692,1749404350000
dbfs:/FileStore/tables/ejemplo.txt,ejemplo.txt,189,1748748188000
dbfs:/FileStore/tables/links.csv,links.csv,197979,1749404504000
dbfs:/FileStore/tables/movies.csv,movies.csv,494431,1749404505000
dbfs:/FileStore/tables/padron_limpio.csv/,padron_limpio.csv/,0,0
dbfs:/FileStore/tables/por_ciclo_2016_2018.csv,por_ciclo_2016_2018.csv,904931,1749404351000
dbfs:/FileStore/tables/ratings.csv,ratings.csv,2483723,1749404506000


# Manejo y Optimización de Particiones en Spark

En este notebook aprenderás a:
- Visualizar y modificar el número de particiones de un DataFrame.
- Usar las funciones `repartition` y `coalesce`.
- Guardar datos con particionado por columna y medir el impacto en el rendimiento.
Trabajaremos con el dataset MovieLens.


In [0]:
# Cargar datos MovieLens (ratings.csv) suponiendo que está en /dbfs/FileStore/movielens/ratings.csv
# Tabla Ratings
ratings_schema  = StructType(fields=[
    StructField("userId",IntegerType(),True), 
    StructField("movieId",IntegerType(),True),
    StructField("rating",DecimalType(precision=2,scale=1),True),
    StructField("timestamp",LongType(),True)
])
ratingsDf = spark.read\
    .option("header", True)\
    .option("dateFormat", "yyyyMMdd")\
    .schema(ratings_schema)\
    .csv("dbfs:/FileStore/tables/ratings_full.csv")\
    .withColumn(\
            "date",\
            from_unixtime("timestamp", "yyyyMMdd"))\
                .drop('timestamp')



In [0]:
ratingsDf.printSchema()


root
 |-- userId: integer (nullable = true)
 |-- movieId: integer (nullable = true)
 |-- rating: decimal(2,1) (nullable = true)
 |-- date: string (nullable = true)



In [0]:
ratingsDf.count()



Out[7]: 33832162

In [0]:
# Ver número de particiones
print("Particiones iniciales:", ratingsDf.rdd.getNumPartitions())



Particiones iniciales: 8


In [0]:
%time
# Cambiar el número de particiones a 16
df_repart = ratingsDf.repartition(16)
print("Después de repartition(16):", df_repart.rdd.getNumPartitions())



Después de repartition(16): 16


In [0]:
%time
# Reducir a 2 particiones con coalesce
df_coalesce = df_repart.coalesce(2)
print("Después de coalesce(2):", df_coalesce.rdd.getNumPartitions())



Después de coalesce(2): 2


In [0]:
%%sh
ls -las /tmp 
mkdir /tmp/rating_by_rating

total 88
4 drwxrwxrwt 1 root root 4096 Jun  8 20:18 .
4 drwxr-xr-x 1 root root 4096 Jun  8 17:32 ..
4 drwxrwxrwt 2 root root 4096 Jun  8 17:32 .ICE-unix
4 drwxrwxrwt 2 root root 4096 Jun  8 17:32 .Test-unix
4 drwxrwxrwt 2 root root 4096 Jun  8 17:32 .X11-unix
4 drwxrwxrwt 2 root root 4096 Jun  8 17:32 .XIM-unix
4 drwxrwxrwt 2 root root 4096 Jun  8 17:32 .font-unix
4 drwxr-xr-x 3 root root 4096 Jun  8 17:34 Rserv
4 drwx------ 2 root root 4096 Jun  8 17:38 RtmpnkHKVk
4 -rw-r--r-- 1 root root   22 Jun  8 17:32 chauffeur-daemon-params
4 -rw-r--r-- 1 root root    4 Jun  8 17:32 chauffeur-daemon.pid
4 -rw-r--r-- 1 root root  156 Jun  8 17:32 chauffeur-env.sh
4 -rw-r--r-- 1 root root 1042 Jun  8 17:32 custom-spark.conf
4 -rw-r--r-- 1 root root   19 Jun  8 17:33 driver-daemon-params
4 -rw-r--r-- 1 root root    4 Jun  8 17:33 driver-daemon.pid
4 -rw-r--r-- 1 root root 3321 Jun  8 17:33 driver-env.sh
4 drwxr-xr-x 2 root root 4096 Jun  8 17:33 hsperfdata_root
4 drwxr-xr-x 2 root root 4096 Jun  8 

In [0]:
%time
# Guardar datos particionando por 'rating'
import time
start = time.time()
output_path = "/tmp/rating_by_rating"
ratingsDf.write.mode("overwrite").partitionBy("rating").parquet(output_path)
print("Tiempo:", time.time() - start, "segundos")


CPU times: user 4 µs, sys: 0 ns, total: 4 µs
Wall time: 6.91 µs
Tiempo: 104.14708518981934 segundos


In [0]:
%%sh
ls -las /tmp/rating_by_rating/

total 8
4 drwxr-xr-x 2 root root 4096 Jun  8 20:18 .
4 drwxrwxrwt 1 root root 4096 Jun  8 20:18 ..


In [0]:
print("Particiones originales:", ratingsDf.rdd.getNumPartitions())



Particiones originales: 8


In [0]:
def medir_tiempo_particiones(df, num_particiones):
    df_mod = df.repartition(num_particiones)
    print(f"\nUsando {num_particiones} particiones:")
    start = time.time()
    # Operación costosa: groupBy + count
    resultado = df_mod.groupBy("movieId").count().collect()
    print("Tiempo:", round(time.time() - start, 3), "segundos")



In [0]:
for n in [2, 4, 8, 16, 32]:
    medir_tiempo_particiones(ratingsDf, n)


Usando 2 particiones:
Tiempo: 63.93 segundos

Usando 4 particiones:
Tiempo: 63.58 segundos

Usando 8 particiones:
Tiempo: 64.058 segundos

Usando 16 particiones:
Tiempo: 65.2 segundos

Usando 32 particiones:
Tiempo: 66.018 segundos
